In [ ]:
#| default_exp display


In [ ]:
#| export
from __future__ import annotations


# Display
> Display utils for notebooks.

# Prologue

In [ ]:
#| export
import json
import uuid
from typing import Mapping
from typing import overload
from typing import Sequence

import fastcore.all as FC
import rich
from IPython.display import display
from IPython.display import DisplayHandle
from IPython.display import HTML


In [ ]:
from olio.common import setup_console
from rich.markdown import Markdown


----


In [ ]:
console, cprint = setup_console(140)

----

# Rich display

In [ ]:
def display_json(json):
    from rich.json import JSON
    from rich.jupyter import display as rich_display
    json_renderable = JSON.from_data(json)
    a = list(console.render(json_renderable))
    rich_display(a, console._render_buffer(a))

In [ ]:
display_json({'a': 1, 'b': 2})

{
  "a": 1,
  "b": 2
}

In [ ]:
#| export

@overload
def pretty_repr(*o, html:bool=True, text:bool=False, **kwargs) -> str: ...
@overload
def pretty_repr(*o, html:bool=False, text:bool=True, **kwargs) -> str: ...
def pretty_repr(*o, html:bool=True, text:bool=True, **kwargs) -> dict[str, str]|str:
    from rich.pretty import Pretty
    d = Pretty(*o, **kwargs)._repr_mimebundle_(
        include=((),('text/plain',))[text] + ((),('text/html',))[html], 
        exclude=((),('text/plain',))[not text] + ((),('text/html',))[not html]
        )
    return d if len(d) > 1 else tuple(d.values())[0]

In [ ]:
display(HTML(pretty_repr({'a': 1, 'b': [1,2,3]}, text=False)))
print(pretty_repr({'a': 1, 'b': [1,2,3]}, html=False))
cprint({'a': 1, 'b': [1,2,3]})

{'a': 1, 'b': [1, 2, 3]}



{'a': 1, 'b': [1, 2, 3]}

In [ ]:
#| export

def rich_display(*o, dhdl: DisplayHandle|None=None):
    if not o: return
    vv:tuple[str, ...] = tuple(FC.flatten([_.items() for _ in map(pretty_repr, o)]))  # type: ignore
    dd = {'text/plain':'\n'.join(vv[1::4]), 'text/html':'\n'.join(vv[3::4])}
    if dhdl: dhdl.update(dd, raw=True)
    else: display(dd, raw=True)

In [ ]:
rich_display({'a': 1, 'b': 2}, [3, 4, 5])

{'a': 1, 'b': 2}

[3, 4, 5]

In [ ]:
dhdl = display(display_id=True)
rich_display({'a': 1, 'b': 2}, [3, 4, 5], dhdl=dhdl)


{'a': 1, 'b': 2}

[3, 4, 5]

# Collapsable JSON

In [ ]:
#| export

class RenderJSON(object):
    def __init__(self, json_data, max_height=200, init_level=0):
        if isinstance(json_data, Sequence):
            s = json.dumps(list(json_data))
        elif isinstance(json_data, Mapping):
            s = json.dumps(dict(json_data))
        elif hasattr(json_data, 'to_dict'):
            s = json.dumps(json_data.to_dict())
        elif hasattr(json_data, 'to_json'):
            s = json_data.to_json()
        else:
            s = json_data
        self.json_str = s
        self.uuid = str(uuid.uuid4())
        self.max_height = max_height
        self.init_level = init_level

    def display(self):
        html_content = f"""
<div id="wrapper-{self.uuid}" style="width: 100%; max-height: {self.max_height}px; overflow-y: auto;">
    <div id="{self.uuid}" style="width: 100%;"></div>
    <script>
        function renderMyJson() {{
            renderjson.set_show_to_level({self.init_level});
            document.getElementById('{self.uuid}').appendChild(renderjson({self.json_str}));
        }};
        function loadRenderjson() {{
            if (window.renderjson) return Promise.resolve();
            return new Promise((resolve, reject) => {{
                const script = document.createElement('script');
                script.src = 'https://cdn.jsdelivr.net/npm/renderjson@latest/renderjson.js';
                script.onload = resolve;
                script.onerror = reject;
                document.head.appendChild(script);
            }});
        }};
        loadRenderjson().then(renderMyJson).catch(err => console.error('Failed to load renderjson:', err));
    </script>
</div>
"""
        display(HTML(html_content), metadata={'bridge': {'skip':True}})

    def _ipython_display_(self):
        self.display()

In [ ]:
json_data = {
    "name": "Petronila",
    "age": 28,
    "interests": ["reading", "cycling", "technology"],
    "education": {
        "bachelor": "Computer Science",
        "master": "Data Science",
        "phd": "Not enrolled"
    }
}

RenderJSON(json_data, init_level=1).display()

# Colophon
----


In [ ]:
#| eval: false

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
#| eval: false

if FC.IN_NOTEBOOK:
    nb_path = '17_display.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
